In [1]:
import pdfplumber

def pdf_to_txt(pdf_path, txt_path):
    with pdfplumber.open(pdf_path) as pdf, open(txt_path, "w", encoding="utf-8") as txt_file:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                txt_file.write(text + "\n")
            else:
                txt_file.write("[No text found on this page]\n")

# Usage
pdf_to_txt("input/fabric-fundamentals.pdf", "fabric.txt")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [ ]:
import asyncio
import logging
from dotenv import load_dotenv
from graphrag.config.models.graph_rag_config import GraphRagConfig
from graphrag.config.enums import IndexingMethod
from graphrag.index.run.run_pipeline import run_pipeline
from graphrag.index.run.utils import create_callback_chain
from graphrag.callbacks.reporting import create_pipeline_reporter
from graphrag.callbacks.workflow_callbacks import WorkflowCallbacks
from graphrag.logger.null_progress import NullProgressLogger
from graphrag.index.workflows.factory import PipelineFactory
from graphrag.index.typing.pipeline_run_result import PipelineRunResult
from graphrag.logger.base import ProgressLogger

import yaml
import os

# Load environment variables from .env file explicitly
load_dotenv()

async def build_index(
    config: GraphRagConfig,
    method: IndexingMethod = IndexingMethod.Standard, # other option fast
    is_update_run: bool = False,
    memory_profile: bool = False,
    callbacks: list[WorkflowCallbacks] | None = None,
    progress_logger: ProgressLogger | None = None,
) -> list[PipelineRunResult]:
    logger = progress_logger or NullProgressLogger()
    callbacks = callbacks or []
    callbacks.append(create_pipeline_reporter(config.reporting, None))

    workflow_callbacks = create_callback_chain(callbacks, logger)

    outputs: list[PipelineRunResult] = []

    pipeline = PipelineFactory.create_pipeline(config, method, is_update_run)
    workflow_callbacks.pipeline_start(pipeline.names())

    async for output in run_pipeline(
        pipeline,
        config,
        callbacks=workflow_callbacks,
        logger=logger,
        is_update_run=is_update_run,
    ):
        outputs.append(output)
        if output.errors and len(output.errors) > 0:
            logger.error(output.workflow)
        else:
            logger.success(output.workflow)
        logger.info(str(output.result))

    workflow_callbacks.pipeline_end(outputs)
    return outputs

def load_settings(settings_path: str, input_folder: str) -> GraphRagConfig:
    with open(settings_path, "r") as f:
        settings_dict = yaml.safe_load(f)

    # Update the input root dynamically here
    settings_dict['input']['root'] = input_folder

    return GraphRagConfig(**settings_dict)

async def main(input_folder: str):
    logging.basicConfig(level=logging.INFO)

    config = load_settings("settings.yaml", input_folder)

    results = await build_index(
        config,
        method=IndexingMethod.Standard,
        memory_profile=False
    )

    for result in results:
        print(f"Workflow: {result.workflow}")
        if result.errors:
            print(f"Errors: {result.errors}")
        else:
            print("Successfully indexed.")

# Ensure input folder exists and settings.yaml is correct
input_folder_path = "./input"
os.makedirs(input_folder_path, exist_ok=True)

await main(input_folder_path)


/opt/conda/envs/ai-agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:graphrag.storage.file_pipeline_storage:Creating file storage at /workspaces/chatui/notebooks/output
INFO:graphrag.index.input.factory:loading input from root_dir=input
INFO:graphrag.index.input.factory:using file storage for input
INFO:graphrag.storage.file_pipeline_storage:search /workspaces/chatui/notebooks/input for files matching .*\.txt$
INFO:graphrag.index.input.util:Found 1 InputFileType.text files, loading 1
INFO:graphrag.index.input.util:Total number of unfiltered InputFileType.text rows: 1
INFO:graphrag.index.run.run_pipeline:Final # of rows loaded: 1
INFO:graphrag.utils.storage:reading table from storage: documents.parquet
INFO:graphrag.utils.storage:reading table from storage: documents.parquet
INFO:graphrag.utils

Workflow: create_base_text_units
Successfully indexed.
Workflow: create_final_documents
Successfully indexed.
Workflow: extract_graph
Successfully indexed.
Workflow: finalize_graph
Successfully indexed.
Workflow: create_communities
Successfully indexed.
Workflow: create_final_text_units
Successfully indexed.
Workflow: create_community_reports
Successfully indexed.
Workflow: generate_text_embeddings
Successfully indexed.
